In [27]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

username = "artia_readonly"
password = 'ap3ljax#!0frack'
host = "artia-dashboard-psql.postgres.database.azure.com"
port = "5432"              
database = "artia_dashboard"

url = URL.create(
    "postgresql+psycopg2",
    username=username,
    password=password,
    host=host,
    port=port,
    database=database,
)

engine = create_engine(url)

In [28]:
coverage_tags = pd.read_sql("SELECT * FROM coverage_tags", engine)
state_coverages = pd.read_sql("SELECT * FROM state_coverages", engine)
state_coverage_coverage_tags = pd.read_sql("SELECT * FROM state_coverage_coverage_tags", engine)
drugs = pd.read_sql("SELECT * FROM drugs", engine)
states = pd.read_sql("SELECT * FROM states", engine)

In [29]:
state_coverage_coverage_tags.head()

,state_coverage_id,coverage_tag_id
0,10539,1383
1,464,1377
2,487,1377
3,489,1377
4,483,1377


In [62]:
states

,id,code,name,pool_id,is_pharmacy_carve_out,has_statewide_pdl,pdl_vendor,pos_vendor,dur_vendor,pa_vendor,...,detailed_agenda,dur_info_allowed,orphan_oncology_review,icer_indicator,cost_effectiveness_information_drug_reviews,icer_indicator_source,cost_effectiveness_information_drug_reviews_source,general_non_pdl_step_therapy_count,general_pa_step_therapy_count,is_mcou_state_controlled
0,46,VT,Vermont,2,False,False,Change Healthcare,Change Healthcare,Change Healthcare,Change Healthcare,...,False,None,False,False,None,None,None,1+,1+,False
1,42,SD,South Dakota,2,False,False,State,Optum,Acentra,Acentra,...,False,None,False,False,None,None,None,1+,1+,False
2,29,NV,Nevada,1,False,False,Prime,Prime,Prime,Prime,...,False,None,False,False,None,None,None,2,2,False
3,50,WI,Wisconsin,3,True,False,Prime,Gainwell,Acentra,Acentra,...,False,None,False,False,None,None,None,1+,1+,True
4,32,NM,New Mexico,4,False,False,No PDL,Conduent,Conduent,Conduent,...,False,None,False,False,None,None,None,None,None,False
5,35,ND,North Dakota,2,True,False,Change Healthcare,Conduent,Acentra,Acentra,...,False,None,False,False,None,None,None,1+,1+,True
6,19,LA,Louisiana,3,False,True,Prime,Gainwell,Gainwell,University of LA-Monroe/Gainwell,...,False,None,False,False,None,None,None,1+,1+,True
7,41,SC,South Carolina,1,False,True,Prime,Prime,Prime/USC,Prime/USC,...,False,None,False,False,None,None,None,2,2,True
8,45,UT,Utah,2,False,False,Change Healthcare,Change Healthcare,University of Utah/State,State,...,False,None,False,False,None,None,None,1+,1+,False
9,13,ID,Idaho,3,False,False,Prime,Prime,Prime,State,...,False,None,False,False,None,None,None,1+,2,False


In [66]:
coverage_data = pd.DataFrame()

state_name_map = states.set_index('id')['name'].to_dict()
coverage_data['State'] = state_coverages['state_id'].map(state_name_map)

state_map = states.set_index('id')['code'].to_dict()
coverage_data['ST'] = state_coverages['state_id'].map(state_map)

drug_map = drugs.set_index('id')['name'].to_dict()
coverage_data['Product'] = state_coverages['drug_id'].map(drug_map)

coverage_tag_map = coverage_tags.set_index('id')['name'].to_dict()
coverage_tag_array = state_coverage_coverage_tags['coverage_tag_id'].map(coverage_tag_map)

coverage_tag_df = pd.DataFrame()
coverage_tag_df['coverage_tag'] = coverage_tag_array
coverage_tag_df['state_coverage_id'] = state_coverage_coverage_tags['state_coverage_id']

state_coverage_coverage_tag_map = (
    coverage_tag_df
    .groupby('state_coverage_id')['coverage_tag']
    .apply(lambda x: ', '.join(sorted(set(x.dropna()))))
    .to_dict()
)
coverage_data['Coverage Tag'] = state_coverages['id'].map(state_coverage_coverage_tag_map)

coverage_data = coverage_data.sort_values('State').reset_index(drop=True)

In [67]:
coverage_data[coverage_data.Product == 'Lonhala Magnair']

,State,ST,Product,Coverage Tag
270,Alabama,AL,Lonhala Magnair,"inhalation solution, Refill Kit"
278,Alabama,AL,Lonhala Magnair,"inhalation solution, Starter Kit"
2858,Alaska,AK,Lonhala Magnair,"inhalation solution, Starter Kit"
2865,Alaska,AK,Lonhala Magnair,"inhalation solution, Refill Kit"
4129,Arizona,AZ,Lonhala Magnair,"inhalation solution, Starter Kit"
...,...,...,...,...
74386,West Virginia,WV,Lonhala Magnair,"inhalation solution, Refill Kit"
75841,Wisconsin,WI,Lonhala Magnair,"inhalation solution, Starter Kit"
75851,Wisconsin,WI,Lonhala Magnair,"inhalation solution, Refill Kit"
76369,Wyoming,WY,Lonhala Magnair,"inhalation solution, Refill Kit"
